In [12]:
# -*- coding: utf-8 -*-
"""
Created on Sun Dec 24 16:40:40 2021
@author: Arya
"""
import sys
import csv
import nltk
import numpy
import sklearn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse


In [ ]:
# for files in glob.glob("5501_files/*.txt"):
#     print (files)

def textReader(fname):
    doc=open(fname,'rb')
    document = ' '.join(line.decode('utf-8').strip() for line in doc.readlines())
    return document

In [ ]:
documents=[]
for files in glob.glob("5501_files/*.txt"):
     documents.append(textReader(files))
    
     
nltk_tokenizer=RegexpTokenizer(r'\w+')

for index,document in enumerate(documents):
    documents[index]= nltk_tokenizer.tokenize(document)
    
stop_words=set(stopwords.words("english"))
for index,document in enumerate(documents):
    documents[index]=[w for w in document if w not in stop_words ]

ps=PorterStemmer()

In [ ]:
all_words=[]
for index,document in enumerate(documents):
    list_of_map = list(map(lambda x: ps.stem(x).lower(),document))
    documents[index] = list_of_map
    all_words = all_words + documents[index]

# print (document)

In [ ]:
all_words=nltk.FreqDist(all_words)
print (all_words.most_common(30))    

all_words.plot(30)
# from IPython.display import Image
# Image(filename='C:\Users\Arya\Desktop\jjjjjjtttttt\plot.png')

col=14000
word_features= [w for (w,c) in all_words.most_common(col)]

In [ ]:
def find_Features(document):
    words=document
    features=[]
    for w in word_features:
        features.append(words.count(w))
    return features

In [13]:
def entropy(p):
	return -(p * numpy.log2(p) + (1-p) * numpy.log2((1-p)))
# entropy(0.95)

In [14]:
def relative_entropy(p, q):
    return sum(p[i] * numpy.log2(p[i]/q[i]) for i in range(len(p)))
# relative_entropy([0.95,0.05], [0.2,0.8])

In [ ]:
# index = word_features.index('iran')
# print('The index of iran is:', index)
# index = word_features.index('teacher')
# print('The index of teacher is:', index)

In [ ]:
def termDocumentMatrix(documents):
    feature_sets=np.zeros((len(documents),col))
    i = 0
    for document in documents:
        feature_sets[i,]= find_Features([word for word in document])
        i=i+1
        
    return pd.DataFrame(feature_sets,columns = word_features)

In [ ]:
tdm = termDocumentMatrix(documents)
print ("**** Term Document Matrix ****")

print (tdm)

print(tdm.T[0])
print(len(tdm.T[0]))    

mi = []
# for t in iran_co:
#     print(word_features[t])

In [ ]:
for j in range(0, 5500):
    mi.append(sklearn.metrics.mutual_info_score(tdm.T[1079], tdm.T[j]))
    print(mi)
iran_co = np.argpartition(mi, -10)[-10:]
print(iran_co)

for t in iran_co:
    print(word_features[t])

In [ ]:
# for i in range (0, 10):
#     for j in range (0, 10):
#         mi = round(sklearn.metrics.mutual_info_score(tdm.T[i], tdm.T[j]), 4)
#         print(mi)

# mi = sklearn.metrics.mutual_info_score(tdm.T[0], tdm.T[5])
# print("Mutual Information Score is:")
# print(mi)

In [ ]:
mi2 = sklearn.metrics.mutual_info_score(tdm.T[2], tdm.T[3])
print("Mutual Information Score is:")
print(mi2)

In [ ]:
def __init__(self, b=0.75, k1=1.6):
        self.vectorizer = TfidfVectorizer(norm=None, smooth_idf=False)
        self.b = b
        self.k1 = k1

    def fit(self, X):
        self.vectorizer.fit(X)
        y = super(TfidfVectorizer, self.vectorizer).transform(X)
        self.avdl = y.sum(1).mean()

    def transform(self, q, X):
        b, k1, avdl = self.b, self.k1, self.avdl

        X = super(TfidfVectorizer, self.vectorizer).transform(X)
        len_X = X.sum(1).A1
        q, = super(TfidfVectorizer, self.vectorizer).transform([q])
        assert sparse.isspmatrix_csr(q)

        X = X.tocsc()[:, q.indices]
        denom = X + (k1 * (1 - b + b * len_X / avdl))[:, None]
        # idf(t) = log [ n / df(t) ] + 1 in sklearn, so it need to be coneverted
        # to idf(t) = log [ n / df(t) ] with minus 1
        idf = self.vectorizer._tfidf.idf_[None, q.indices] - 1.
        numer = X.multiply(np.broadcast_to(idf, X.shape)) * (k1 + 1)                                                          
        return (numer / denom).sum(1).A1


In [ ]:
'''
************************************************
similarity_matrix=pd.DataFrame(np.corrcoef(tdm.T),index=word_features,columns=word_features)
np.savetxt('newPicksData.txt', similarity_matrix)

print ("**** Similarity Matrix ****")
print (str(round(similarity_matrix, 3)))
************************************************
'''